# Setup

In [1]:
# --- Setup ---
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

In [2]:
# Display settings
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [3]:
# --- File paths ---
input_file = 'data/processed/hotelrec_final.csv'
output_file = 'data/processed/hotelrec_cleaned.csv'
chunk_size = 1_000_000

In [4]:
# --- Load first chunk to preview ---
reader = pd.read_csv(input_file, chunksize=chunk_size)
first_chunk = next(reader)
print("Preview of first chunk:")
display(first_chunk.head())

Preview of first chunk:


,hotel_id,hotel_name,hotel_location,author,date,rating,sentiment_score,sleep quality,value,rooms,service,cleanliness,location
0,1121769,Hotel Baltic,Giulianova Province of Teramo Abruzzo,violettaf340,2019-01,5.0,0.341327,NaN,NaN,NaN,NaN,NaN,NaN
1,1121769,Hotel Baltic,Giulianova Province of Teramo Abruzzo,Lagaiuzza,2016-01,5.0,0.272183,NaN,NaN,NaN,NaN,NaN,NaN
2,1121769,Hotel Baltic,Giulianova Province of Teramo Abruzzo,ashleyn763,2014-10,5.0,0.475000,NaN,5.0,NaN,5.0,NaN,5.0
3,1121769,Hotel Baltic,Giulianova Province of Teramo Abruzzo,DavideMauro,2014-08,5.0,0.623636,5.0,NaN,NaN,5.0,5.0,NaN
4,1121769,Hotel Baltic,Giulianova Province of Teramo Abruzzo,Alemma11,2013-08,4.0,0.218607,3.0,4.0,4.0,5.0,3.0,4.0


In [5]:
# --- Function to count nulls in chunks ---
def count_nulls_in_chunks(file_path, chunk_size):
    null_counts = None
    for chunk in pd.read_csv(file_path, chunksize=chunk_size):
        if null_counts is None:
            null_counts = chunk.isnull().sum()
        else:
            null_counts += chunk.isnull().sum()
    return null_counts

# --- Function to count duplicated in chunks ---
def count_duplicates_in_chunks(file_path, chunk_size):
    total_duplicates = 0
    for chunk in pd.read_csv(file_path, chunksize=chunk_size):
        total_duplicates += chunk.duplicated().sum()
    return total_duplicates

# Preprocess

In [6]:
# --- Initial null count before cleaning ---    [~1 min]
print("Initial missing values per column:")
initial_nulls = count_nulls_in_chunks(input_file, chunk_size)
print(initial_nulls)

Initial missing values per column:
hotel_id                  0
hotel_name                0
hotel_location           20
author                23889
date                      0
rating                    0
sentiment_score           0
sleep quality      27672279
value              23877794
rooms              24847780
service            14691088
cleanliness        23858431
location           24743058
dtype: int64


In [ ]:
print("\nInitial duplicate rows count:")        # [~1min 15s]
print(count_duplicates_in_chunks(input_file, chunk_size))


Initial duplicate rows count:
164


In [ ]:
# Estimate number of lines (excluding header)       [~8s]
with open(input_file, 'r', encoding='utf-8') as f:
    total_lines = sum(1 for _ in f) - 1

num_chunks = (total_lines // chunk_size) + 1
num_chunks

51

In [ ]:
# [~4 min 10s]
import os
#import pandas as pd
#from tqdm import tqdm

# --- Setup ---
current_dir = os.getcwd()
file_path = os.path.join(current_dir, 'data', 'raw', 'hotelrec_final.csv')
output_path = os.path.join(current_dir, 'data', 'raw', 'hotelrec_final_cleaned.csv')

chunk_size = 1_000_000
seen_rows = set()
first_chunk = True

# --- Get the total number of rows for progress ---
# (quickly without loading the whole file)
total_rows = sum(1 for line in open(file_path)) - 1  # minus header
total_chunks = (total_rows // chunk_size) + 1

# --- Reader ---
reader = pd.read_csv(file_path, chunksize=chunk_size)

# --- Process with tqdm progress bar ---
for chunk in tqdm(reader, total=total_chunks, desc="Dropping Duplicates", mininterval=1):
    # Drop duplicates WITHIN chunk
    chunk = chunk.drop_duplicates().copy()

    # Drop duplicates ACROSS chunks
    chunk['row_hash'] = chunk.apply(lambda row: hash(tuple(row)), axis=1)
    chunk = chunk[~chunk['row_hash'].isin(seen_rows)]

    # Update seen rows
    seen_rows.update(chunk['row_hash'])

    # Drop helper column
    chunk = chunk.drop(columns=['row_hash'])

    # Write to cleaned output file
    if first_chunk:
        chunk.to_csv(output_path, index=False, mode='w')
        first_chunk = False
    else:
        chunk.to_csv(output_path, index=False, header=False, mode='a')

print(f"\nDone. Cleaned data saved to: {output_path}")


Cleaning chunks: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 51/51 [04:13<00:00,  4.98s/chunk]


Done. Cleaned data saved to: data/processed/hotelrec_cleaned.csv


In [ ]:
# --- Null count after cleaning ---     [~2min 10s]
print("\nMissing values in cleaned data:")
cleaned_nulls = count_nulls_in_chunks(output_file, chunk_size)
print(cleaned_nulls)

print("\nDuplicate rows after cleaning:")
print(count_duplicates_in_chunks(output_file, chunk_size))


Missing values in cleaned data:
hotel_id                  0
hotel_name                0
hotel_location           20
author                23889
date                      0
rating                    0
sentiment_score           0
sleep quality      27672224
value              23877772
rooms              24847754
service            14691066
cleanliness        23858408
location           24743032
dtype: int64

Duplicate rows after cleaning:
0


In [ ]:
#[~6min]
import gzip

# File paths
csv_path = 'data/processed/hotelrec_cleaned.csv'
gz_path = 'data/processed/hotelrec_cleaned.csv.gz'

# Compress using gzip
with open(csv_path, 'rb') as f_in:
    with gzip.open(gz_path, 'wb') as f_out:
        f_out.writelines(f_in)

print(f"Compressed file saved to: {gz_path}")

Compressed file saved to: data/processed/hotelrec_cleaned.csv.gz
